In [1]:
from qiskit import *
from numpy.random import randint, shuffle
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute, assemble
from qiskit.visualization import plot_histogram, plot_bloch_vector
from qiskit.visualization import plot_state_qsphere, plot_state_city, plot_bloch_multivector
from qiskit.visualization import array_to_latex

# numpy for linear algebra stuff
import random
from numpy.random import randint, shuffle
import numpy as np
import matplotlib.pyplot as plt

# To supress the deprecation warnings
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

In [2]:
def generate_bits(N):
    bits = randint(0, 2, N)
    return bits

In [3]:
def encode_message(bits, bases):
    message = []
    for i in range(len(bits)):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [4]:
def encode_bits(bits, bases):
    l = len(bits)
    base_circuit = QuantumCircuit(l, l)
    for i in range(l):
        if bases[i] == 0:
            if bits[i] == 1:
                base_circuit.x(i)
        if bases[i] == 1:
            if bits[i] == 0:
                base_circuit.h(i)
            if bits[i] == 1:
                base_circuit.x(i)
                base_circuit.h(i)
    base_circuit.barrier()
    return base_circuit

In [5]:
def eavesdropping(qubits, N, ERROR_RATE = 0.5) :
#     e = randint((2 * MAX_ERROR_RATE * N) // 100 + 1)
#     e = int((2 * MAX_ERROR_RATE * N) // 100) + 1

    e = int(ERROR_RATE * N)
#     print("Eve measures ", e, " qubits")
    
    circ = qubits.copy()
    rand_pos = sorted(random.sample(range(N), e))
   
    #Eve measures selected signal with a randomly chosen basis
    eve_bases = [-999]*N
    for pos in rand_pos :
        random_gate = randint(2)
        eve_bases[pos] = random_gate
        if random_gate == 0:
            circ.measure(pos, pos)
        else :
            circ.h(pos)
            circ.measure(pos, pos)
    circ.barrier()
    backend = Aer.get_backend('qasm_simulator')
    m = execute(circ, backend, shots=1, memory = True).result().get_counts()
    bits = [int(ch) for ch in list(m.keys())[0]][::-1]
    
    return circ, eve_bases

In [6]:
def received_encode(qubits,N,ERROR_RATE=0.5,Eve=False):
    if Eve and ERROR_RATE !=0:
        return eavesdropping(qubits, N, ERROR_RATE)
    else:
        return qubits,[-999]*N

In [7]:
def measure_bits(circuit, bases, agreed_base_indices):
    backend = Aer.get_backend('qasm_simulator')
    for j in range(len(bases)):
        if bases[j] == 0:
            circuit.measure(j,j)
        if bases[j] == 1:
            circuit.h(j)
            circuit.measure(j,j)
    r = execute(circuit, backend, shots=1, memory = True).result().get_counts()
    return circuit, [int(ch) for ch in list(r.keys())[0]][::-1]

In [8]:
def measure_bits_withsifting(circuit, bases, agreed_base_indices):
    backend = Aer.get_backend('qasm_simulator')
    for j in range(len(bases)):
        if j in agreed_base_indices:
            if bases[j] == 0:
                circuit.measure(j,j)
            if bases[j] == 1:
                circuit.h(j)
                circuit.measure(j,j)
    r = execute(circuit, backend, shots=1, memory = True).result().get_counts()
    print(r)
    return circuit, [int(ch) for ch in list(r.keys())[0]][::-1]

In [9]:
def sifting(alice_bases,bob_bases):
    agreed_base_indices = [i for i in range(len(alice_bases)) if alice_bases[i] == bob_bases[i]]
    return agreed_base_indices

In [10]:
def qber(alice_bits,bob_bits,agreed_base_indices,frac=0.5):
    test_len = int(len(agreed_base_indices)*frac) + 1
    test_base_indices = sorted(random.sample(agreed_base_indices,test_len))
    
    alice_test_bits = []
    bob_test_bits = []
    for i in test_base_indices:
        alice_test_bits.append(alice_bits[i])
        bob_test_bits.append(bob_bits[i])
    
    num_error = len([j for j in range(len(alice_test_bits)) if alice_test_bits[j] != bob_test_bits[j]])
    error_rate = num_error / len(alice_test_bits)
    
    return error_rate, test_base_indices

In [11]:
def error_rate(alice_bits,bob_bits):
    num_error = len([j for j in range(len(alice_bits)) if alice_bits[j] != bob_bits[j]])
    error_rate = num_error / len(alice_bits)
    return error_rate